<a href="https://colab.research.google.com/github/YasJanam/RL1/blob/main/ReinforceTrainer_4/ReinforceTrainer_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Requirements**

##### install modules

In [ ]:
#!pip uninstall -y gym gymnasium box2d box2d-py

In [ ]:
!pip install gymnasium==0.29.1
!pip install swig
!pip install box2d-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 41.9 MB/s eta 0:00:00
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 1.2.1
    Uninstalling gymnasium-1.2.1:
      Successfully uninstalled gymnasium-1.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.8-cp312-cp312-linux_x86_64.whl size=2409500 sha256=532ec855ba55df00267c6f99d6a27c226727c1ed53f775166abfbd843262b911
  Stored in directory: /root/.cache/pip/wheels/d6/3c/ab/b6fd75459cadc56f4a4125d4cb387a708a59ca8589e4cc6b7d
Successf

##### imports

In [ ]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### **Environments**

##### **CartPole**

In [ ]:
cartpole = gym.make('CartPole-v1')

##### **MountainCar**

حالت : دو بعدی
( موقعیت ، سرعت )

عمل ها : جلو / عقب / بدون حرکت

پاداش :
تا وقتی بالا نره، پاداش منفی میگیره

هدف : بالا بردن ماشین از تپه

In [ ]:
MountainCar = gym.make('MountainCar-v0')

In [ ]:
print(MountainCar.observation_space)
print(MountainCar.observation_space.shape)
print(MountainCar.action_space)

Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
(2,)
Discrete(3)


##### **LunarLander**

In [ ]:
LunarLander = gym.make('LunarLander-v2')

#### **PolicyNet**

In [ ]:
class PolicyNet(nn.Module):
  def __init__(self,state_dim,action_dim,hidden_dim=100):
    super(PolicyNet,self).__init__()
    self.fc = nn.Sequential(
        nn.Linear(state_dim,hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim,action_dim),
        nn.Softmax(dim=-1)
    )

  def forward(self,x):
    return self.fc(x)

#### **ReinforceTrainer**

In [ ]:
class ReinforceTrainer(nn.Module):
  def __init__(self,env,hidden_dim=128,lr=0.01,gamma=0.99,episodes_num=1000,log_interval=100,max_rewards=float('inf')):
    super().__init__()
    self.env = env
    self.lr = lr
    self.log_interval = log_interval
    self.policy_hidden_dim = hidden_dim
    self.episode_num = episodes_num
    self.gamma = gamma
    self.max_rewards = max_rewards
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n
    self.policy = PolicyNet(state_dim=state_dim,action_dim=action_dim,hidden_dim=self.policy_hidden_dim).to(device)
    self.optimizer = optim.Adam(self.policy.parameters(),lr=self.lr)

  def compute_returns(self,rewards):
    G = 0
    returns = []
    for r in reversed(rewards):
      G = r + self.gamma * G
      returns.insert(0,G)
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + 1e-8)  # normalization
    return returns.to(device)

  def train(self):
    for episode in range(self.episode_num):
      state, _ = self.env.reset()
      log_probs = []
      rewards = []
      done = False

      # rollout
      while not done:
        state = torch.tensor(state, dtype=torch.float32).to(device)

        probs = self.policy(state)
        dist = Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)

        next_state, reward, terminated, truncated, info = self.env.step(action.item())
        done = terminated or truncated

        log_probs.append(log_prob)
        rewards.append(reward)
        state = next_state

      # log
      total_reward = sum(rewards)
      if episode % self.log_interval == 0 or episode == (self.episode_num -1) :
        print(f"Episode {episode}, Total Reward: {total_reward}")

      # early stopping
      if total_reward >= self.max_rewards:
        print(f"Reached max reward {total_reward:.2f}, stopping training!")
        break

      # policy update
      returns = self.compute_returns(rewards)

      loss = 0
      for log_prob, G in zip(log_probs, returns):
        loss -= log_prob * G

      self.optimizer.zero_grad()
      loss.backward()
      self.optimizer.step()

      self.env.close()

**DONE** ( gymnasium-done ) :

terminated :
اپیزود به خاطر رسیدن به هدف یا شکست تموم شده

truncated :
اپیزود به خاطر محدودیت زمانی قطع شده

done = terminated or truncated

**سیگنال پاداش معنی دار** :

الگوریتم رینفورس برای یاد گیری نیاز به پاداش معنی دار داره ! یعنی یه تفاوتی بین اکشن های خوب و بد باشه. چون اینجوری کار میکنه که بر حسب پاداش هایی که میگیره به سمت هدف حرکت کنه

**MountainCar** :

در این محیط تا وقتی عامل به هدف نرسد تمام پاداش ها یکی هستند و برابر -1 اند. یعنی پاداش یک سیگنال معنی دار که عامل را به سمت هدف ببرد نیست. در نتیجه الگوریتم رینفورس روی این محیط کارایی ندارد. در ادامه این موضوع قابل مشاهده است



#### **Test**

In [ ]:
def test(policy,env):
  state, _ = env.reset()
  done = False
  total_reward = 0

  while not done:
      state_tensor = torch.tensor(state, dtype=torch.float32)

      # انتخاب اکشن با policy
      with torch.no_grad():  # gradient محاسبه نشه
          probs = policy(state_tensor)
      dist = torch.distributions.Categorical(probs)
      action = dist.sample().item()

      # اعمال اکشن
      next_state, reward, terminated, truncated, _ = env.step(action)
      done = terminated or truncated

      # جمع‌آوری پاداش
      total_reward += reward
      state = next_state

  return total_reward


In [ ]:
def tests(policy,env,num_tests):
  total_rewards = []
  for _ in range(num_tests):
    total_reward = test(policy,env)
    total_rewards.append(total_reward)
  rewards = [float(x) for x in total_rewards]
  return rewards

#### **tarin**

##### CartPole

In [ ]:
cp_trainer = ReinforceTrainer(env=cartpole,hidden_dim=15,lr=0.001,gamma=0.99,episodes_num=1000)
cp_trainer.train()

Episode 0, Total Reward: 24.0
Episode 100, Total Reward: 12.0
Episode 200, Total Reward: 34.0
Episode 300, Total Reward: 21.0
Episode 400, Total Reward: 47.0
Episode 500, Total Reward: 89.0
Episode 600, Total Reward: 63.0
Episode 700, Total Reward: 38.0
Episode 800, Total Reward: 41.0
Episode 900, Total Reward: 270.0
Episode 999, Total Reward: 350.0


##### MountainCar

In [ ]:
mc_trainer = ReinforceTrainer(env=MountainCar,hidden_dim=128,lr=0.003,gamma=0.99,episodes_num=500,log_interval=100)
mc_trainer.train()

Episode 0, Total Reward: -200.0
Episode 100, Total Reward: -200.0
Episode 200, Total Reward: -200.0
Episode 300, Total Reward: -200.0
Episode 400, Total Reward: -200.0
Episode 499, Total Reward: -200.0


برابر 200 است rollout حداکثر تعداد استپ در یک

ملاحظه میکنید که پاداش کل تمام اپیزود ها برابر 200- شده است

شده اند truncate ها  rollout این یعنی تمام

##### LunarLander

In [ ]:
trainer = ReinforceTrainer(env=LunarLander,hidden_dim=50,lr=0.006,gamma=0.99,episodes_num=2000,log_interval=80)
trainer.train()

Episode 0, Total Reward: -53.22990772696299
Episode 80, Total Reward: -71.50219906093626
Episode 160, Total Reward: -27.5168907254766
Episode 240, Total Reward: -2.2126424281705965
Episode 320, Total Reward: 79.14784516288864
Episode 400, Total Reward: 288.1995739594064
Episode 480, Total Reward: 25.65213221542885
Episode 560, Total Reward: 283.59043685217114
Episode 640, Total Reward: 240.02758369313892
Episode 720, Total Reward: 280.8945688418088
Episode 800, Total Reward: -31.120080271048337
Episode 880, Total Reward: 265.86016334329076
Episode 960, Total Reward: 296.78013480048537
Episode 1040, Total Reward: 256.0518604122203
Episode 1120, Total Reward: 253.05118451874563
Episode 1200, Total Reward: -5.715695042433794
Episode 1280, Total Reward: 239.9295319837459
Episode 1360, Total Reward: 214.14864132132612
Episode 1440, Total Reward: 186.44925894713276
Episode 1520, Total Reward: 300.8294054036445
Episode 1600, Total Reward: 274.1881272949603
Episode 1680, Total Reward: 259.7948

In [ ]:
trainer = ReinforceTrainer(env=LunarLander,hidden_dim=18,lr=0.005,gamma=0.99,episodes_num=2500,log_interval=150,max_rewards=300)
trainer.train()

Episode 0, Total Reward: -230.7038690052147
Episode 150, Total Reward: -71.84500730422391
Episode 300, Total Reward: 10.499415628912885
Episode 450, Total Reward: 32.67165617267992
Episode 600, Total Reward: 102.42998149664243
Episode 750, Total Reward: 126.93486553183588
Episode 900, Total Reward: 167.79275608654623
Episode 1050, Total Reward: 131.97596996515577
Episode 1200, Total Reward: 31.478092511944794
Episode 1350, Total Reward: 37.563974672560676
Episode 1500, Total Reward: 150.0603448661049
Episode 1650, Total Reward: 34.741153431294265
Episode 1800, Total Reward: 43.35619766601775
Episode 1950, Total Reward: 247.83937042013017
Episode 2100, Total Reward: 101.761291494854
Episode 2250, Total Reward: 245.54383611534453
Episode 2400, Total Reward: 121.17526064434972
Episode 2499, Total Reward: 130.06700893816534


In [ ]:
results = tests(trainer.policy,LunarLander,10)
results

[130.17978966071044,
 133.21017342542714,
 127.35077283499963,
 224.66911780281038,
 242.53399819138812,
 210.18660229356198,
 147.67705959752644,
 193.74221652603677,
 262.3148771569899,
 65.51015671118907]

In [ ]:
trainer = ReinforceTrainer(env=LunarLander,hidden_dim=30,lr=0.003,gamma=0.99,episodes_num=3000,log_interval=150,max_rewards=300)
trainer.train()

Episode 0, Total Reward: -107.94190874546008
Episode 150, Total Reward: -184.6184893474179
Episode 300, Total Reward: -341.1119807891307
Episode 450, Total Reward: -107.46065396086138
Episode 600, Total Reward: -204.50831404637873
Episode 750, Total Reward: 56.172122688986235
Episode 900, Total Reward: 46.205336856350954
Episode 1050, Total Reward: -199.40439431398102
Episode 1200, Total Reward: 68.70085134722672
Episode 1350, Total Reward: 239.4702302677389
Episode 1500, Total Reward: 148.86400117910765
Episode 1650, Total Reward: 21.03401594737157
Episode 1800, Total Reward: 63.621154168926495
Episode 1950, Total Reward: 34.590147386591156
Episode 2100, Total Reward: 48.388518283107686
Episode 2250, Total Reward: 163.28472630734692
Episode 2400, Total Reward: 116.11142987434333
Episode 2550, Total Reward: 172.76827586634118
Episode 2700, Total Reward: 102.33545557140775
Episode 2850, Total Reward: 10.99155536487065
Episode 2999, Total Reward: 172.30461738394482


In [ ]:
results = tests(trainer.policy,LunarLander,10)
results

[140.4280540498257,
 181.28223141047064,
 255.57986375522742,
 3.6549266744151083,
 117.25934289255697,
 152.5821752521934,
 169.6383866076964,
 147.9446069529386,
 144.59193761266326,
 161.65569661646225]